# Decarbonization Projcet


# Model Implementation

#### Preparations

Import packages

In [128]:
from gurobipy import *
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

Load parameters


In [129]:
parameters = pd.read_csv("data.csv", index_col='source')
parameters = parameters.astype("float64")
parameters

,number,plant_capacity_power_kw,plant_capacity_force_kwh,plant_generate_force_kwh,fixed_cost_power_dollar_kW,fixed_cost_plant_dollar,operating_cost(dollar_kWh),revenues(dollor_kWh),co2(pounds_kWh),capacity_national_kwh,capacity_national_kw,generation_national_kwh,load_factor,capacity_%,generation_%
source,,,,,,,,,,,,,,,
coal,8.0,860112.50,7.534586e+09,1.725150e+09,750.0,6.450844e+08,0.03,0.32,2.26,6.027668e+10,6880900.0,1.380120e+10,0.23,0.08,0.06
natural gas,48.0,244968.75,2.145926e+09,4.730250e+08,600.0,1.469812e+08,0.07,0.32,0.97,1.030045e+11,11758500.0,2.270520e+10,0.22,0.14,0.10
CCGT,23.0,655143.48,5.739057e+09,2.245357e+09,900.0,5.896291e+08,0.07,0.32,0.77,1.319983e+11,15068300.0,5.164320e+10,0.39,0.17,0.23
nuclear,4.0,827450.00,7.248462e+09,6.399750e+09,3100.0,2.565095e+09,0.04,0.32,0.00,2.899385e+10,3309800.0,2.559900e+10,0.88,0.04,0.12
hydro,739.0,37715.83,3.303907e+08,7.711177e+07,3100.0,1.169191e+08,0.01,0.32,0.00,2.441587e+11,27872000.0,5.698560e+10,0.23,0.32,0.26
wind,256.0,58180.08,5.096575e+08,1.478203e+08,3100.0,1.803582e+08,0.01,0.32,0.00,1.304723e+11,14894100.0,3.784200e+10,0.29,0.17,0.17
solar,141.0,45094.33,3.950263e+08,5.525532e+07,4500.0,2.029245e+08,0.01,0.32,0.00,5.569871e+10,6358300.0,7.791000e+09,0.14,0.07,0.04


In [130]:
parameters["capacity_national_kw"]

source
coal            6880900.0
natural gas    11758500.0
CCGT           15068300.0
nuclear         3309800.0
hydro          27872000.0
wind           14894100.0
solar           6358300.0
Name: capacity_national_kw, dtype: float64

In [131]:
num_total_years = 75
num_decision_years = 10
unit_emission = np.array(parameters["co2(pounds_kWh)"])
print("Shape of unit emission:", unit_emission.shape)
unit_cost = np.array(parameters["operating_cost(dollar_kWh)"])
fixed_cost = np.array(parameters["fixed_cost_plant_dollar"])
yearly_demand = np.array([sum(parameters["generation_national_kwh"]) for i in range(num_total_years)])
unit_price = 0.32
capacity_force = np.array(parameters["plant_capacity_force_kwh"]) # kwh for one plant for one year
# generate_plant = np.array(parameters["plant_generate_force_kwh"])
num_plant_start = np.array(parameters["number"]) # number of plants at year 0
budget_start = 9080*10**6 # million of euros
discount_carbon = 0.04
discount_money = 0.1
co2_price = 20000 / 2000

Shape of unit emission: (7,)


Set up index sets

In [132]:
10**6

1000000

In [133]:
unit_emission

array([2.26, 0.97, 0.77, 0.  , 0.  , 0.  , 0.  ])

In [134]:
post_years = range(num_decision_years, num_total_years)  # 10-74
sources = range(unit_emission.shape[0])  # 7
decision_years= range(0, num_decision_years) # 0-9
all_years = range(0, num_total_years)

#### Set up model

In [135]:
m = Model()
m.setParam('PoolSolutions',50)
m.setParam('NonConvex', 2)

Set parameter PoolSolutions to value 50
Set parameter NonConvex to value 2


Decision variables

In [136]:
dv_num_plant = m.addVars(sources, decision_years, vtype=GRB.INTEGER, lb=0.0, name="num_plant")  # 7*10
dv_add_plant = m.addVars(sources, decision_years, vtype=GRB.INTEGER, lb=0.0, name="add_plant")  # 7*10
dv_minus_plant = m.addVars(sources, decision_years, vtype=GRB.INTEGER, lb=0.0, name="minus_plant")  # 7*10
dv_generate_plant = m.addVars(sources, all_years, lb=0.0, name="generate_plant")
dv_yearly_budget = m.addVars(decision_years, name="budget") # 10

Objective function

In [137]:
# Primary Objective: NPV carbon , set negative in accordance with the MAXIMIZE model sense
sum_co2_cost_before = (sum(unit_emission[j] * dv_generate_plant[j,i] * dv_num_plant[j, i] * co2_price/(1 + discount_money)**(i+1) for i in decision_years for j in sources))
sum_co2_cost_after = (sum(unit_emission[j] * dv_generate_plant[j,i] * dv_num_plant[j, num_decision_years-1] * co2_price/(1 + discount_money)**(i+1) for i in post_years for j in sources))
sum_co2_cost = sum_co2_cost_before + sum_co2_cost_after

# Objective 2: total profit = revenue - fixed cost - operating cost
sum_revenue_before = sum(dv_generate_plant[j,i]*dv_num_plant[j, i]*unit_price/(1 + discount_money)**(i+1) for i in decision_years for j in sources) 
sum_revenue_after = sum(dv_generate_plant[j,i]*dv_num_plant[j, num_decision_years-1]*unit_price/(1 + discount_money)**(i+1) for i in post_years for j in sources) 
sum_revenue = sum_revenue_before + sum_revenue_after

sum_fixed_cost = sum(fixed_cost[j]*dv_add_plant[j, i]/(1 + discount_money)**(i+1) for j in sources for i in decision_years)

sum_operating_cost_before = sum(unit_cost[j]*dv_generate_plant[j,i]*dv_num_plant[j, i]/(1 + discount_money)**(i+1) for j in sources for i in decision_years)
sum_operating_cost_after = sum(unit_cost[j]*dv_generate_plant[j,i]*dv_num_plant[j, num_decision_years-1]/(1 + discount_money)**(i+1) for j in sources for i in post_years)
sum_operating_cost = sum_operating_cost_before + sum_operating_cost_after

m.setObjective(sum_revenue - sum_fixed_cost - sum_operating_cost - sum_co2_cost)
m.modelSense = GRB.MAXIMIZE
m.setParam('PoolSolutions',20)


Set parameter PoolSolutions to value 20


Constraints

In [138]:
num_plant_start

array([  8.,  48.,  23.,   4., 739., 256., 141.])

In [139]:
# Definition of S = s0 + sum(x)
for i in decision_years:
    for j in sources:
        m.addConstr(dv_num_plant[j, i] == num_plant_start[j] + sum(dv_add_plant[j, k] - dv_minus_plant[j, k] for k in range(0, i)))

# Budget Definition
for i in decision_years:
    if i == 0:
        m.addConstr(dv_yearly_budget[i] <= budget_start)
    else:
        yearly_cost = sum(fixed_cost[j]*dv_add_plant[j, i-1] + unit_cost[j]*dv_generate_plant[j,i-1]*dv_num_plant[j, i-1] for j in sources)
        profit_prev_year = yearly_demand[i-1]*unit_price - yearly_cost
        m.addConstr(dv_yearly_budget[i] <= profit_prev_year / (1 + discount_money))

# Budget
for i in decision_years:
    # fixed + operating cost
    yearly_cost = sum(fixed_cost[j]*dv_add_plant[j, i] + unit_cost[j]*dv_generate_plant[j,i]*dv_num_plant[j, i] for j in sources)
    m.addConstr(yearly_cost <= dv_yearly_budget[i])

# Demand vs Capacity
for i in decision_years:
    m.addConstr(sum(capacity_force[j] * dv_num_plant[j, i] for j in sources) >= yearly_demand[i])

# Demand vs Generate
for i in decision_years:
    m.addConstr(sum(dv_generate_plant[j,i]*dv_num_plant[j, i] for j in sources) == yearly_demand[i])
for i in post_years:
    m.addConstr(sum(dv_generate_plant[j,i]*dv_num_plant[j, num_decision_years-1] for j in sources) == yearly_demand[i])

# Generate vs Capacity
for i in all_years:
    for j in sources:
        m.addConstr(dv_generate_plant[j,i] <= capacity_force[j])

# Can't demolish clean energy
for j in range(3, 7):
    for i in decision_years:
        m.addConstr(dv_minus_plant[j, i] == 0)

# Diversity, if one energy fails, the capacity of other sources should satisfy 75% of demand
for i in decision_years:
    for j in sources:
        m.addConstr(sum(dv_generate_plant[j,i]*dv_num_plant[j, i] for j in sources) - dv_generate_plant[j,i]*dv_num_plant[j, i]>= 0.75*yearly_demand[i])

# Can't build nuclear any more
for i in decision_years:
    m.addConstr(dv_add_plant[3, i] == 0)


#### Solve the model

In [140]:
# Solve
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 656 rows, 745 columns and 1346 nonzeros
Model fingerprint: 0x0f32e6ac
Model has 525 quadratic objective terms
Model has 164 quadratic constraints
Variable types: 535 continuous, 210 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+09]
  QMatrix range    [9e-03, 1e+00]
  QLMatrix range   [1e+00, 3e+09]
  Objective range  [5e+07, 2e+09]
  QObjective range [4e-04, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+00, 2e+11]
  QRHS range       [6e+10, 2e+11]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 587 rows and 77 columns
Presolve time: 0.00s
Presolved: 1983 rows, 1113 columns, 5701 nonzeros
Presolved model has 444 bilinear constraint(s)
Variable types: 984 continuous, 129 integer (0 binary)

Root relaxation: objectiv

It takes 0.67 seconds to solve the model.

In [141]:
# get the set of variables
x = m.getVars()

# Ensure status is optimal
assert m.Status == GRB.Status.OPTIMAL

# Query number of multiple objectives, and number of solutions
nSolutions  = m.SolCount
nObjectives = m.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')


Problem has 1 objectives
Gurobi found 5 solutions


In [144]:
# For each solution, print value of first three variables, and
# value for each objective function
solutions = []
for s in range(nSolutions):
  # Set which solution we will query from now on
  m.params.SolutionNumber = s

  # Print objective value of this solution in each objective
  print('Solution', s, ':', end='')
  
  for j in range(70):
    print(x[j].VarName, x[j].Xn, end='')
  print('')

  # query the full vector of the o-th solution
  solutions.append(m.getAttr('Xn',x))


Solution 0 :num_plant[0,0] 8.0num_plant[0,1] -0.0num_plant[0,2] -0.0num_plant[0,3] -0.0num_plant[0,4] -0.0num_plant[0,5] -0.0num_plant[0,6] -0.0num_plant[0,7] -0.0num_plant[0,8] -0.0num_plant[0,9] -0.0num_plant[1,0] 48.0num_plant[1,1] -0.0num_plant[1,2] -0.0num_plant[1,3] -0.0num_plant[1,4] -0.0num_plant[1,5] -0.0num_plant[1,6] -0.0num_plant[1,7] -0.0num_plant[1,8] -0.0num_plant[1,9] -0.0num_plant[2,0] 23.0num_plant[2,1] 8.0num_plant[2,2] 6.0num_plant[2,3] 6.0num_plant[2,4] 6.0num_plant[2,5] 6.0num_plant[2,6] 6.0num_plant[2,7] 6.0num_plant[2,8] 5.0num_plant[2,9] 5.0num_plant[3,0] 4.0num_plant[3,1] 4.0num_plant[3,2] 4.0num_plant[3,3] 4.0num_plant[3,4] 4.0num_plant[3,5] 4.0num_plant[3,6] 4.0num_plant[3,7] 4.0num_plant[3,8] 4.0num_plant[3,9] 4.0num_plant[4,0] 739.0num_plant[4,1] 739.0num_plant[4,2] 739.0num_plant[4,3] 739.0num_plant[4,4] 739.0num_plant[4,5] 739.0num_plant[4,6] 739.0num_plant[4,7] 739.0num_plant[4,8] 739.0num_plant[4,9] 739.0num_plant[5,0] 256.0num_plant[5,1] 256.0num_plan